# Introducción a las cadena en LangChain.

En este capítulo vamos a contemplar los siguientes aspectos:

* ¿Cómo crear cadenas para construir aplicaciones complejas?
  
* ¿Cómo crear un modelo secuencial para combinar cadenas?

* ¿Cómo enrutar a la mejor cadena?

* ¿Cómo crear cadenas avanzadas de transformación y preguntas/respuestas sobre nuestros datos vectorizados?

##  Cadenas de forma secuencial.
```{index} LLMChain
```

Las cadenas nos permiten combinar múltiples componentes para crear una aplicación única y coherente. Por ejemplo, podemos crear una cadena que tome la entrada del usuario , la formatee con PromptTemplate y luego pase la respuesta formateada a un LLM . Podemos construir cadenas más complejas combinando varias cadenas o combinando cadenas con otros componentes, por tanto, las cadenas nos permite vincular la salida de una llamada
LLM con la entrada de otra llamada LLM.

Las cadenas tienen un componente básico conocido como objeto LLMChain . Podemos pensar en LLMChain como una simple llamada a LLM que tendrá una entrada (normalmente el prompt o solicitud) y una salida (el resultado).

Si encadenamos varios LLMChain podemos tener un modelo secuencial, el modelo secuencial simple tiene solo 1 entrada global y 1 salida global, no nos proporciona los resultados intermedios. Cada objeto LLMChain intermedio solo tiene 1 entrada y 1 salida, por tanto la salida de 1.

El tema de manejo de cadenas en LangChain, lo podemos encontrar en el siguiente enlace:

https://python.langchain.com/v0.1/docs/modules/chains/

Veamos a continuación un ejemplo de cadena secuencial

In [3]:
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate

chat = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)


## Creación de un objeto de tipo LLMCahain.

Veamos a continuación cómo se construyen estos objetos

In [4]:
human_message_prompt = HumanMessagePromptTemplate.from_template(
        "Dame un nombre de compañía que sea simpático para una compañía que fabrique {producto}"
    )

chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

print(chain.invoke(input="Lavadoras"))

C:\Users\Francisco\AppData\Local\Temp\ipykernel_2152\305437190.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=chat_prompt_template)


{'producto': 'Lavadoras', 'text': '¡Claro! Aquí te presento algunas opciones de nombres de compañía que podrían ser simpáticos para una empresa que fabrica lavadoras:\n\n1. **Lavandería Happy**: Un nombre que evoca felicidad y satisfacción, lo que se relaciona directamente con el uso de la lavadora.\n2. **Sudor Libre**: Un nombre que se refiere al sudor humano que se elimina después de hacer la lavadora, y que tiene un sonido alegre y juguetón.\n3. **LavaEasy**: Un nombre que sugiere fácilmente y comodidad, lo que es qué muchas personas buscan al utilizar una lavadora.\n4. **La Lavandera**: Un nombre que evoca la idea de cuidado y atención personalizada, lo que se relaciona con la calidad de las lavadoras fabricadas por la compañía.\n5. **BlancoBrillo**: Un nombre que sugiere brillo y pureza, lo que se refiere a la limpieza y el efecto de la lavadora en los ropa blanqueados.\n6. **La Compañera Perfecta**: Un nombre que sugiere confiabilidad y fiabilidad, lo que es qué las personas busc

Como vemos la ejecución del código anterior nos devuelve un warning, indicando que en un futuro esa construcción no se va a permitir. En concreto el warning recibido es el siguiente:

```
LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=chat_prompt_template)
```

Esto lo podemos ver en la documentación de Langchain, en este enlace:

https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html

A continuación procedemos a modificar las instrucciones anteriores para adaptarlas a lo que será permitido en el futuro

In [5]:
chain = chat_prompt_template | chat
print(chain.invoke(input="Lavadoras"))

content='¡Claro! Aquí te dejo algunas sugerencias de nombres de compañías que podrían ser simpáticas y relevantes para una empresa que se especializa en fabricación de lavadoras:\n\n1. **Lava con Amor**: Este nombre refleja la emoción y el cuidado que una persona puede sentir cuando lava ropa, lo que puede conectar con los clientes y generar confianza en la marca.\n2. **Fiesta Líquida**: Esta nameplay sobre la idea de "fiesta líquida" (agua) puede ser divertido y alegre, transmitiendo una sensación de alegría y movilidad.\n3. **La Luna Lavadora**: La luna es un símbolo de pureza y frescura, lo que encaja bien con la fonction del producto. Esto puede generar una imagen positiva y relajante para la marca.\n4. **Lava de Oro**: Este nombre sugiere algo valoroso y especializado, lo que puede crear confianza y estabilidad en los clientes.\n5. **Vida Limpia**: Este nombre enfatiza la idea de vivir saludablemente y manteniendo las cosas limpias, lo cual es muy atractivo para personas que valor

## Cadena secuencial simple.

Veamos a continuación un ejemplo de concatenación de cadenas simples, en el sentido de que el resultado de una cadena es utilizado para ejecutar la siguiente.

In [7]:
from langchain.chains.sequential import SimpleSequentialChain

In [8]:
llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

In [9]:
template = "Dame un simple resumen con un listado de puntos para un post de un blog acerca de {tema}"
prompt1 = ChatPromptTemplate.from_template(template)
chain_1 = LLMChain(llm=llm,prompt=prompt1)

In [10]:
template = "Escribe un post completo usando este resumen: {resumen}"
prompt2 = ChatPromptTemplate.from_template(template)
chain_2 = LLMChain(llm=llm,prompt=prompt2)

In [12]:
full_chain = SimpleSequentialChain(chains=[chain_1,chain_2],
                                  verbose=True) #verbose=True nos irá dando paso a paso lo que hace, pudiendo ver los resultados intermedios

Ahora vamos a introducir como primer parámetro "Inteligencia Artifical", entonces en base a la primera cadena nos devolverá una serie de conceptos sobre esta materia y esta salida alimentará a la segunda cadena para obtener el resultado final 

In [13]:
result = full_chain.invoke(input="Inteligencia Artificial")



> Entering new SimpleSequentialChain chain...
**¡Claro! Aquí te presento un resumen y un listado de puntos para un post sobre Inteligencia Artificial:**

**¿Qué es la Inteligencia Artificial?**

La Inteligencia Artificial (IA) se refiere a la capacidad de una máquina para simular la inteligencia humana, aprendiendo y adaptándose a nuevos datos y situaciones. La IA ha revolucionado muchas áreas, desde la robótica hasta los sistemas de recomendación en línea.

**Ventajas de la Inteligencia Artificial:**

* **Aumento eficiencia**: La IA puede automatizar tareas repetitivas y mejorar la productividad
* **Mejora de la precisión**: La IA reduce errores humanos y ofrece respuestas más exactas
* **Personalización**: La IA puede analizar comportamientos y preferencias para ofrecer experiencias personalizeadas

**Desafíos y limitaciones de la Inteligencia Artificial:**

* **Seguridad**: La IA puede ser vulnerable a ataques cibernéticos y pérdida de privacidad
* **Ética**: La IA debe ser diseña

In [15]:
print(result['output'])

**¡Descubre el potencial de la Inteligencia Artificial en tu vida!**

La Inteligencia Artificial (IA) es una tecnología que se refiere a la capacidad de una máquina para simular la inteligencia humana, aprendiendo y adaptándose a nuevos datos y situaciones. En las últimas décadas, la IA ha revolucionado muchas áreas, desde la robótica hasta los sistemas de recomendación en línea.

**¿Qué es la Inteligencia Artificial?**

La IA se enfoca en crear algoritmos y modelos que puedan aprender y mejorar su desempeño a medida que se confrontan con nuevos datos. Esto se logra mediante el uso de tecnologías como procesamiento de lenguaje natural, aprendizaje automático y reconocimiento de patrones.

**Ventajas de la Inteligencia Artificial**

La IA ha revolucionado muchos ámbitos en nuestra vida, ofreciendo varias ventajas significativas:

*   **Aumento eficiencia**: La IA puede automatizar tareas repetitivas y mejorar la productividad, lo que nos permite enfocarnos en tareas más complejas.
*   *

## Construcción modelo secuencial completo.

Este tipo de cadenas, son muy similares a las vistas en el anterior apartado, pero ofrecen la ventaja de que nos permiten tener acceso a todas las salidas del LLMChains internas. Para ver su funcionamiento, vamos a ver un caso de uso de rendimiento de empleados.

Imaginemos que estamos en el departamento de recursos humanos y queremos hacer un plan de mejora para todas las funciones que tiene cada uno de los empleados de la empresa.
Lo que tendremos de partida es una revisión del rendimiento del empleado y vamos a considerar un modelo secuencial completo. En el primer bloque de LLMcahin, se va a especializar en obtener un resumen a partir de la revisión que tenemos como entrada, el siguiente bloque va a estar especializado en detectar las debilidades a partir del resumen anterior y por último se nos va a proporcionar un plan de mejora basándose en las debilidades que se han detectado. Esto se hará para todos los empleados de la plantilla de una manera casi totalmente automática.

El esquema de este proceso se muestra en la siguiente figura:
![](fig/secuencialCompleto.PNG)

Veamos cómo implementamos esto mediante código

In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain #importamos el SequentialChain que es el modelo completo

llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)


Creamos la primera cadena. El parámetro va a ser "revision_rendimiento". Establecemos un *output_key*, esto es importante proque así tenemos un acceso directo al resultado que obtenemos de esta cadena.

In [3]:
template1 = "Dame un resumen del rendimiento de este trabajador:\n{revision_rendimiento}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = LLMChain(llm=llm,
                     prompt=prompt1,
                     output_key="resumen_revision")

#Opciones objetos runnables: chain_1= prompt1 | llm

El *output_key* que aquí utilizamos va a ser utilizado como entrada de la siguiente cadena. Ahora creamos la segunda cadena (chain2). En este caso el parámetro que utilizamos en el template2, tiene que llamarse exactamente igual que como se ha denominado al *output_key* establecido en el paso anterior. En este segunda eslabón a la salida la llamamos *debilidades*.

In [4]:
template2 = "Identifica las debilidades de este trabajador dentro de de este resumen de la revisión:\n{resumen_revision}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = LLMChain(llm=llm,
                     prompt=prompt2,
                     output_key="debilidades")

#Opciones objetos runnables: chain_2= prompt2 | llm

Pasamos ahora al paso tres. Para el caso del parámetro de ete paso cabe decir lo mismo que se ha comentado anteriormente.

In [6]:
template3 = "Crea un plan de mejora para ayudar en estas debilidades:\n{debilidades}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = LLMChain(llm=llm,
                     prompt=prompt3,
                     output_key="plan_mejora")

#Opciones objetos runnables: chain_3= prompt3 | llm

Creamos ahora el secuentialchain.

In [7]:
seq_chain = SequentialChain(chains=[chain_1,chain_2,chain_3],
                            input_variables=['revision_rendimiento'],
                            output_variables=['resumen_revision','debilidades','plan_mejora'],
                            verbose=True)

Vamos a crear la variable de entrada, que para este ejemplo simplemente es una variable tipo carácter con un determinado valor. En la práctica esto puede ser una llamada a una base de datos interna o una base de datos vactorial.

In [8]:
revision_rendimiento_empleado = '''
Revisión de Rendimiento del Empleado

Nombre del Empleado: Juan Pérez
Posición: Analista de Datos
Período Evaluado: Enero 2023 - Junio 2023

Fortalezas:
Juan ha demostrado un fuerte dominio de las herramientas analíticas y ha proporcionado informes detallados y precisos que han sido de gran ayuda para la toma de decisiones estratégicas. Su capacidad para trabajar en equipo y su disposición para ayudar a los demás también han sido notables. Además, ha mostrado una gran ética de trabajo y una actitud positiva en el entorno laboral.

Debilidades:
A pesar de sus muchas fortalezas, Juan ha mostrado áreas que necesitan mejoras. En particular, se ha observado que a veces tiene dificultades para manejar múltiples tareas simultáneamente, lo que resulta en retrasos en la entrega de proyectos. También ha habido ocasiones en las que la calidad del trabajo ha disminuido bajo presión. Además, se ha identificado una necesidad de mejorar sus habilidades de comunicación, especialmente en lo que respecta a la presentación de datos complejos de manera clara y concisa a los miembros no técnicos del equipo. Finalmente, se ha notado una falta de proactividad en la búsqueda de soluciones a problemas imprevistos, confiando a menudo en la orientación de sus superiores en lugar de tomar la iniciativa.
'''

In [9]:
results = seq_chain.invoke(revision_rendimiento_empleado)



> Entering new SequentialChain chain...

> Finished chain.


In [11]:
results #Tenemos un diccionario con todos los resultados intermedios y final

{'revision_rendimiento': '\nRevisión de Rendimiento del Empleado\n\nNombre del Empleado: Juan Pérez\nPosición: Analista de Datos\nPeríodo Evaluado: Enero 2023 - Junio 2023\n\nFortalezas:\nJuan ha demostrado un fuerte dominio de las herramientas analíticas y ha proporcionado informes detallados y precisos que han sido de gran ayuda para la toma de decisiones estratégicas. Su capacidad para trabajar en equipo y su disposición para ayudar a los demás también han sido notables. Además, ha mostrado una gran ética de trabajo y una actitud positiva en el entorno laboral.\n\nDebilidades:\nA pesar de sus muchas fortalezas, Juan ha mostrado áreas que necesitan mejoras. En particular, se ha observado que a veces tiene dificultades para manejar múltiples tareas simultáneamente, lo que resulta en retrasos en la entrega de proyectos. También ha habido ocasiones en las que la calidad del trabajo ha disminuido bajo presión. Además, se ha identificado una necesidad de mejorar sus habilidades de comunic

In [12]:
#Resultado final
print(results['plan_mejora'])

**Plan de Mejora Laboral para Juan Pérez**

**Objetivo:** Mejorar el desempeño laboral de Juan Pérez identificando y abordando sus debilidades en el manejo de múltiples tareas, calidad del trabajo bajo presión, habilidades de comunicación y proactividad en la búsqueda de soluciones.

**Metas a Corto Plazo (6 meses)**

1. **Mejorar el manejo de múltiples tareas**:
 * Establecer y seguir un calendario de tareas y plazos para cada proyecto.
 * Utilizar herramientas de gestión del tiempo como Trello o Asana para organizar y priorizar tareas.
 * Realizar una revisión semestral del progreso y ajustar el plan según sea necesario.
2. **Mejorar la calidad del trabajo bajo presión**:
 * Fomentar la discusión con Juan sobre sus preocupaciones y limitaciones en termos de manejo de proyectos y trabajo bajo presiones.
 * Proporcionar un entorno de trabajo tranquilo en equipo, con una estructura organizada para proyectos.
 * Reestructurar el calendario como máximo para garantizar que no se le pida al

También podemos tener los resultados de los pasos anteriores

In [13]:
#Se puede accceder a los resultados intermedios:
print(results["debilidades"])

Las debilidades identificadas en el resumen de la revisión sobre Juan Pérez son:

1. **Manejo insuficiente de múltiples tareas simultáneamente**: Juan tiene dificultades para gestionar varias tareas al mismo tiempo, lo que puede afectar su productividad y la calidad del trabajo.
2. **Calidad del trabajo bajo presión**: La calidad del trabajo de Juan disminuye cuando se le pide realizar tareas rápidamente o con un plazo límite apretado.
3. **Habilidades de comunicación**: Juan necesitaría mejorar sus habilidades de comunicación tanto para audiencias técnicas como no técnicas, lo que podría afectar su capacidad para trabajar de manera efectiva en equipo y conveyar información claramente a diferentes niveles de la organizacion.
4. **Falta de proactividad en la búsqueda de soluciones**: Juan no siempre actúa con prontitud al detectar problemas o inadecuaciones imprevistas, lo que podría generar retrasos u obstáculos en el trabajo.

En cuanto a las áreas de mejora, es importante mencionar q

## Enrutamiento a cadenas con LLMRouterChain.
```{index} LLMRouterChain
```

LLMRouterChains puede recibir una entrada y redirigirla a la secuencia LLMChain más apropiada.

El enrutador acepta múltiples LLMChains de destino potencial y luego, a través de un mensaje especializado, el enrutador leerá la entrada inicial y generará un diccionario específico que coincida con una de las posibles cadenas de destino para continuar con el procesamiento.

![](fig/enrutador.PNG)

Veamos el siguiente caso de uso

In [14]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain #importamos el SequentialChain que es el modelo completo

llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

### Plantillas de enrutamiento

Generamos dos plantillas de enrutamiento. La primera plantilla sería para un soporte básico a clientes de coches. La seguna plantilla sería para crear el perfil del mecánico.


In [15]:
#Template (prompt) para soporte básico a clientes de coches
plantilla_soporte_basico_cliente = '''Eres una persona que asiste a los clientes de automóviles con preguntas básicas que pueden
necesitar en su día a día y que explica los conceptos de una manera que sea simple de entender. Asume que no tienen conocimiento
previo. Esta es la pregunta del usuario/n{input}'''

In [16]:
#Template (prompt) para soporte avanzados a nuestros expertos en mecánica
plantilla_soporte_avanzado_mecánico = '''Eres un experto en mecánica que explicas consultas avanzadas a los mecánicos
de la plantilla. Puedes asumir que cualquier que está preguntando tiene conocimientos avanzados de mecánica. 
Esta es la pregunta del usuario/n{input}'''

De manera similar, se podrían añadir tantas plantillas como necesitemos.

Ahora creamos el enrutamiento de los prompts, que como podemos ver es una lista de diccionarios.

In [17]:
#Debemos crear una lista de diccionarios, cada diccionario contiene su nombre, la descripción (en base a la cual el enrutador
#hará su trabajo) y el prompt a usar en cada caso
prompt_infos = [
    {'name':'mecánica básica','description': 'Responde preguntas básicas de mecánicas a clientes',
     'prompt_template':plantilla_soporte_basico_cliente},
    {'name':'mecánica avanzada','description': 'Responde preguntas avanzadas de mecánica a expertos con conocimiento previo',
     'prompt_template':plantilla_soporte_avanzado_mecánico},
    
]

### Creación de ConversationChain.

In [18]:
from langchain.chains import LLMChain

In [19]:
#Creamos un diccionario de objetos LLMChain con las posibles cadenas destino
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [21]:
#diccionario de bloques LLMChains
destination_chains

{'mecánica básica': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='Eres una persona que asiste a los clientes de automóviles con preguntas básicas que pueden\nnecesitar en su día a día y que explica los conceptos de una manera que sea simple de entender. Asume que no tienen conocimiento\nprevio. Esta es la pregunta del usuario/n{input}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000196B0E42FD0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000196B0E57CD0>, root_client=<openai.OpenAI object at 0x00000196B0900E10>, root_async_client=<openai.AsyncOpenAI object at 0x00000196B0A84850>, model_name='llama3.2', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://local

In [22]:
#Creamos el prompt y cadena por defecto puesto que son argumento obligatorios que usaremos posteriormente
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

Creamos ahora el *Multi Routing Template*
```{index} Multi Routing Template
```

In [24]:
#Importamos una plantilla que podremos formatear su parámetro {destinations} que tendrá cada nombre y descripción de la información de prompts
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [25]:
print(MULTI_PROMPT_ROUTER_TEMPLATE) #El parámetro importante es {destinations}, debemos formatearlo en tipo string

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

Vamos a establecer todos los posibles destinos del Routing.

In [27]:
#Creamos una string global con todos los destinos de routing usando el nombre y descripción de "prompt_infos"
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
# Veamos el contenido de esta variable que contiene todos los posibles destinos
destinations_str

'mecánica básica: Responde preguntas básicas de mecánicas a clientes\nmecánica avanzada: Responde preguntas avanzadas de mecánica a expertos con conocimiento previo'

### Router Prompt

In [28]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [33]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)


router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str #Formateamos la plantilla con nuestros destinos en la string destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(), #Para transformar el objeto JSON parseándolo a una string
)

In [34]:
print(router_template) #verificamos que se ha formateado correctamente

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
mecánica básica: Responde preguntas básicas de mecánicas a clientes
mecánica avanzada: Responde pregu

### Routing Chain Call

In [35]:
from langchain.chains.router import MultiPromptChain

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, #El objeto con los posibles LLMChain que creamos al inicio
                         default_chain=default_chain, verbose=True #Indicamos el LLMChain por defecto (obligatorio)
                        )

# Vemos que el enrutamiento es correcto y se va al apartado del mecánica básica
chain.invoke("¿Cómo cambio el aceite de mi coche?")

C:\Users\Francisco\AppData\Local\Temp\ipykernel_11380\1508555494.py:3: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,




> Entering new MultiPromptChain chain...
mecánica básica: {'input': '¿Cómo cambiar el aceite de mi coche?'}
> Finished chain.


{'input': '¿Cómo cambiar el aceite de mi coche?',
 'text': '¡Hola! Me alegra poder ayudarte a entender cómo cambiar el aceite de tu coche.\n\n**¿Por qué cambiar el aceite?**\n\nEl aceite del motor es un líquido que ayuda a proteger las partes en movimiento y mantener el motor en buen estado. Con el tiempo, el aceite se impurifica y pierde su eficacia para proteger el motor. Si no lo cambias regularmente, puede causar daños en el frenillo de motor, bombilla de fuelle y otras partes del motor.\n\n**¿Qué necesito para cambiar el aceite?**\n\nPara cambiar el aceite, necesitarás:\n\n* El manual del conductor de tu coche (para saber cuánto aceite está en juego)\n* Un destornillador\n* Una llave de cruz ajustable\n* Un recipiente largo y espejo para ver el interior del motor\n* Aceite fresco para el nuevo motor\n\n**Pasos para cambiar el aceite:**\n\n1. **Apaga el motor**: Asegúrate de que el motor esté frío antes de empezar.\n2. **Limpiar el área**: Limpiando el piso y el suelo alrededor del

In [36]:
# Vemos que el enrutamiento es correcto y se va al apartado del mecánico
chain.invoke("¿Cómo funciona internamente un catalizador?")



> Entering new MultiPromptChain chain...
mecánica avanzada: {'input': '¿Cómo funciona internamente un catalizador? Sin embargo, para que pueda darte una solución específica, ¿podrías decirme qué tipo de sistema de combustión está implementado dentro del catalizador?'}
> Finished chain.


{'input': '¿Cómo funciona internamente un catalizador? Sin embargo, para que pueda darte una solución específica, ¿podrías decirme qué tipo de sistema de combustión está implementado dentro del catalizador?',
 'text': '¡Claro! Me alegra poder ayudarte a entender cómo funcionan los catalizadores.\n\nLos catalizadores son dispositivos muy importantes en la industria automotriz que juegan un papel fundamental en el control de las emisiones de gases nocivos. El sistema de combustión que implementan dentro del catalizador es basado en el proceso de oxidación y reducción, específicamente utilizando catalizadores cerámicos o métlicos con elevadas temperaturas.\n\nHay dos tipos principales de sistemas de combustión utilizados en los catalizadores:\n\n1. **Rack Systems**: Estos catalizadores utilizan un sistema rack-like para maximizar la superficie de exposición de los catalizadores a la corriente del flujo. Esos catalizadores son generalmente cerámicos y tienen una temperatura muy alta (debe 

## Cadenas de transformación.
```{index} TransformChain
```

Las " TransformChain " se refieren a un tipo específico de cadena de transformación que se utiliza para modificar, manipular o transformar datos en el proceso de construcción de aplicaciones basadas en modelos de lenguaje.

Diferentes ejemplo pueden ser los siguientes:

* Normalización de texto: Limpiar y estandarizar el texto de entrada para asegurar consistencia.

* Formateo: Cambiar el formato del texto para adaptarlo a necesidades específicas.

* Filtrado: Eliminar información innecesaria o irrelevante.

El caso de uso que vamos a implementar es un ejemplo de resumir y traducir.

Vamos a obtener información de la wikipedia y lo vamos a meter en un modelo secuencial. Haremos los siguientes pasos:

1.- Tomamos la información que nos llega de wikipedia y lo transformamos sin necesidad de llamar a ningún LLM, con el consiguiente ahorro de costes.

2.- Implementamos una cadena LLM que va a resumir lo del paso anterior.

3.- Por ultimo vamos a traducir el texto.

Veamos cómo implementar todo esto. 

In [2]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SimpleSequentialChain, LLMChain,TransformChain



In [3]:
llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

In [5]:
# importamos los documentos
from langchain.document_loaders import WikipediaLoader
consulta_wikipedia = "atlético de Madrid"
idioma_final = "francés"

In [6]:
loader = WikipediaLoader(query=consulta_wikipedia,lang="es",load_max_docs=10)
data = loader.load()
data[0].page_content

'El Club Atlético de Madrid S. A. D. es un club de fútbol fundado el 26 de abril de 1903. Es uno de los clubes activos más longevos de España. Es el quinto equipo que más temporadas ha estado en Primera División, ha disputado 89 de las 94 ediciones, siendo uno de los diez clubes fundadores de la competición y participantes en la edición inaugural de 1929. En 1991 alcanzó el 3.er puesto en la clasificación histórica del fútbol español, perdiendo dicha posición en el año 2001, recuperándola en 2016 y manteniéndola desde entonces.[5]\u200b Disputa sus encuentros como local en el estadio Metropolitano, con capacidad para 70 692 espectadores.\nIdentificado por sus colores rojos y blancos —de los que recibe el apelativo de «rojiblancos» o «colchoneros»—,[1]\u200b es uno de los clubes españoles más laureados, superando la treintena entre títulos nacionales e internacionales. Con once títulos ligueros, es el tercer club más laureado de la competición. En la competición nacional de Copa del Rey

In [7]:
texto_entrada = data[0].page_content

Defino la función de transformación personalizada

In [8]:
def transformer_function(inputs: dict) -> dict: #Toma de entrada un diccionario y lo devuelve con la transformación oportuna
    texto = inputs['texto']
    primer_parrafo = texto.split('\n')[0]
    return {'salida':primer_parrafo}

In [9]:
transform_chain = TransformChain(input_variables=['texto'],
                                 output_variables=['salida'],
                                 transform=transformer_function)

In [11]:
#Definimos la cadena secuencial
#Creamos bloque LLMChain para resumir
template1 = "Crea un resumen en una línea del siguiente texto:\n{texto}"
prompt = ChatPromptTemplate.from_template(template1)
summary_chain = LLMChain(llm=llm,
                     prompt=prompt,
                     output_key="texto_resumen")

In [12]:
#Creamos bloque LLMChain para traducir
template2 = "Traduce a"+ idioma_final + "el siguiente texto:\n{texto}"
prompt = ChatPromptTemplate.from_template(template2)
#prompt.format_prompt(idioma=idioma_final)
translate_chain = LLMChain(llm=llm,
                     prompt=prompt,
                     output_key="texto_traducido")

In [13]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain,summary_chain,translate_chain],
                                        verbose=True)

In [14]:
result = sequential_chain(texto_entrada)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_8408\2789050324.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = sequential_chain(texto_entrada)




> Entering new SimpleSequentialChain chain...
El Club Atlético de Madrid S. A. D. es un club de fútbol fundado el 26 de abril de 1903. Es uno de los clubes activos más longevos de España. Es el quinto equipo que más temporadas ha estado en Primera División, ha disputado 89 de las 94 ediciones, siendo uno de los diez clubes fundadores de la competición y participantes en la edición inaugural de 1929. En 1991 alcanzó el 3.er puesto en la clasificación histórica del fútbol español, perdiendo dicha posición en el año 2001, recuperándola en 2016 y manteniéndola desde entonces.[5]​ Disputa sus encuentros como local en el estadio Metropolitano, con capacidad para 70 692 espectadores.
"Puedo resumir el texto de la siguiente manera: El Club Atlético de Madrid S.A.D es un club de fútbol fundado en 1903 que ha disputado varias temporadas en la Primera División y mantiene una posición destacada en la clasificación histórica del fútbol español."
Claro, te puedo ayudar con eso. Aquí está el texto 

## Cadenas sobre preguntas y respuestas de los datos.
```{index} Retrieval Augmented Generation RAG, RAG
```

La cadena “load_qa_chain” se utiliza para cargar y configurar una cadena de preguntas y respuestas (Q&A) A). Esta cadena puede combinar múltiples componentes de procesamiento de lenguaje natural para responder preguntas basadas en un contexto específico sin tener que hacerlo manualmente.

La clave es que proporciona una respuesta directa y coherente de la pregunta realizada a partir de nuestra base de datos de vectores en lugar de únicamente devolver el vector de mayor similitud.

Muy común usarla cuando estamos creando un Retrieval Augmented Generation RAG (Generación Aumentada por Recuperación) para obtener información y respuestas basadas en nuestra propia base de datos de vectores.

El planteamiento es el siguiente:

Tenemos una base de datos vectorial y una pregunta del usuario que va a generar una respuesta, esta respuesta se inserta en la cadena QA_Chain para encadenar esa respuesta con el LLM y así obtener una respuesta coherente.

Veamos un ejemplo, primero importamos y generamos el LLM

In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SimpleSequentialChain, LLMChain,TransformChain

In [2]:
llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

Conectamos a una base de datos creada en uno de los apartados anteriores y [que se puede ver en este enlace](almacenamiento).

In [5]:
# conectamos a una base de datos creda en uno de los apartados anteriores
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import SKLearnVectorStore

from langchain_ollama import OllamaEmbeddings

embedding_function = OllamaEmbeddings(
    model="llama3.2",
)

In [6]:
vector_store_connection = SKLearnVectorStore(embedding=embedding_function, persist_path="./BD/ejemplosk_embedding_db", serializer="parquet")

In [7]:
#Cargamos cadena QA
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain #Opción que proporciona también la fuente de datos de la respuesta

In [9]:
chain = load_qa_chain(llm,chain_type='stuff') #chain_type='stuff' se usa cuando se desea una manera simple y directa de cargar y procesar el contenido completo sin dividirlo en fragmentos más pequeños. Es ideal para situaciones donde el volumen de datos no es demasiado grande y se puede manejar de manera eficiente por el modelo de lenguaje en una sola operación.

In [12]:
question = "¿Qué pasó en el siglo de Oro español?"
# Obtengo los párrafos que se adaptan a la pregunta
docs = vector_store_connection.similarity_search(question)
chain.run(input_documents=docs,question=question)

'El Siglo de Oro español fue un período floreciente en las artes y las letras hispanas que abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.\n\nDurante este período, el imperio español experimentó un gran bienestar económico y su influencia se extendió a lo largo del mundo. En el aspecto cultural, hubo una gran fertilidad en las artes plásticas, la literatura, la música y la arquitectura. Muchas expresiones culturales provenientes de España fueron imitadas o adoptadas por otros países europeos.\n\nAlgunos de los logros más destacados del Siglo de Oro español incluyen:\n\n* Miembros de la nobleza y el clero adquirieron riquezas exorbitantes.\n* La pobreza y su existencia también se vuelve más manejable, donde había un mayor movimiento de clases media como por ejemplo, burgueses, comerciantes e ingresaban a los gobiernos.\n* España alcanzó la mayor potencia económica del mundo durante el siglo XVI, con un gran 

## alternativa con el método invoque.


In [11]:
#Estructurar un diccionario con los parámetros de entrada
inputs = {
    "input_documents": docs,
    "question": question
}

chain.invoke(inputs)["output_text"]

'No tengo información detallada sobre lo que paso exactamente durante el siglo de Oro. ¿Podrías proporcionar más contexto o detalles sobre qué quieres destacar? Podemos intentar explorar juntos la historia ese período y ver si podemos encontrar información relevante.'